### Import Modules

<br>

In [1]:
import requests
import json
import time
import itertools
import wget
import os
import pickle
import numpy as np

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy

sns.set_style('white')
import tensorflow as tf
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.callbacks import LearningRateScheduler
from keras import callbacks
from keras import regularizers as reg
from keras.optimizers import SGD, Adam
from keras.models import Sequential , Model
from keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras.initializers as init
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.




<br>
### Open the Preprocessed Poster Data

<br>
Image data is stored in a numpy format and existing on a pickle dump file

<br>

In [2]:
x_train_dict = pickle.load(open('vgg16_training_num.pik' , 'rb'))



<br>
### Specify the training/test split

<br>
 - The data will be split into training and test for validation purpose
 - Data augmentation will be subsequently applied for both training and test
 
 <br>

In [3]:
# Specify the split index
train_split = 8000

# Extract the input arrays from the data container
x_train_raw = x_train_dict['images']

# Apply the test / train split
x_train = np.array(x_train_raw)[:train_split,: ,: , :]
x_test  = np.array(x_train_raw)[train_split:,: ,: , :]

print x_train.shape

(8000, 224, 149, 3)




<br>
### Prepare Data for input

<br>
 - Determine the data axis order according tothe keras configuration
 - Define the input and its attributes
 - Center input data and ensure its type as float 
 - Pre-process labels
 
 <br>
 
** 1 - Arrange Data for Input:**

<br>

In [4]:
# Define the image number of rows
img_rows = x_train.shape[1]

# Dfine the image number of columns
img_cols = x_train.shape[2]

# Re-arrange if channels first configuration
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    
    # Define Input Shape
    input_shape = (3, img_rows, img_cols)

# Re-arrange if channels last configuration
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    
    # Define Input shape
    input_shape = (img_rows, img_cols, 3)

# Define Model Input
input_tensor = Input(shape = (img_rows , img_cols , 3))


<br>
** 2 - Pre-process Data: **

<br>

In [5]:
# Transform to float
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

# Center the data
x_train /= 255.0
x_test  /= 255.0

# Display train Data attributes
print 'x_train shape:', x_train.shape
print  x_train.shape[0], 'train samples'

# Display test data attributes
print 'x_test shape:', x_test.shape
print  x_test.shape[0], 'test samples'

x_train shape: (8000, 224, 149, 3)
8000 train samples
x_test shape: (1988, 224, 149, 3)
1988 test samples



<br>
** 3 - Output Data Pre-processing : **

<br>

In [6]:
# Extract Labels from File
y_raw = pd.read_csv('Genres_labels_All_cleaned.csv')

# Split into train and test
y_train = y_raw.iloc[:, 1:-1].values[:train_split, :]
y_test  = y_raw.iloc[:, 1:-1].values[train_split:, :]

# Define the number of classes
num_classes = y_train.shape[1]

print 'number of classes:  ' , num_classes

number of classes:   14



<br>
### Model Construction

<br>
 - Initiate Data Augmentation
 - Specify any layer regularization parameters
 - Stack the model architecture
 
<br>
** 1 - Initiate data augmentation: **

<br>

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range = 0.5,
    fill_mode = 'wrap')

datagen.fit(x_train)

datagen.fit(x_test)


<br>
** 2 - Initiate Regularization: **

<br>

In [8]:
# Specify regularization parameter
reg_par = 0.01


<br>
** 3 - Import original Model:**

<br>

In [9]:
base_model = VGG16(input_tensor = input_tensor , weights = 'imagenet' , include_top = False)


<br>
** 4 - Stack Model:**

<br>

In [21]:
# ---------- Input  ----------------
x = base_model.output

# ----------- Reshape Layer ---------
x = Flatten(input_shape = base_model.output_shape)(x)

# ------------- Fully Connected --------
x = Dense(128 , activation = 'relu' , kernel_initializer = init.he_normal(109),
          kernel_regularizer = reg.l2(reg_par))(x)

# ------------ Dropout --------------
x = Dropout(0.5)(x)

# ------------- Fully Connected --------
x = Dense(64 , activation = 'relu' , kernel_initializer = init.he_normal(109),
          kernel_regularizer = reg.l2(reg_par))(x)

# ------------ Dropout -----------------
x = Dropout(0.5)(x)

# ------------- Output --------------
mod_out = Dense(num_classes , activation = 'sigmoid')(x)

# --------- Model Packing ----------
model = Model(inputs = base_model.input , outputs = mod_out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 149, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 149, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 149, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 74, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 74, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 74, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 37, 128)       0         
__________


<br>
### Model Compilation and Intiation

<br>
** 1 - Layers Freeze: **

<br>

In [22]:
for layer in base_model.layers:
    layer.trainable = False


<br>
** 2 - Model Compilation : **

<br>

In [23]:
# Specifying Optimization Method
sgd = SGD(lr=0 , momentum = 0.99)

# Model Comilation
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


<br>
** 3 - Training Parameters: **

<br>

In [24]:
# Batch Size
batch_size = 128

# epochs
epochs = 60


<br>
** 4 - Callback Functions: **

<br>

In [25]:
def step_decay(epoch):
    lrate = 0.01
    if epoch > 20 and epoch <= 40:
        lrate = 0.001
    elif epoch > 40:
        lrate = 1e-4
    else:
        lrate = lrate
        
    return lrate

lr = LearningRateScheduler(step_decay)
callbacks_list = [lr]


<br>
### Training Execution

<br>

In [26]:

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),             
                              steps_per_epoch = len(x_train) / batch_size, 
                              epochs=epochs,
                              validation_data = (x_test, y_test),
                              validation_steps = len(x_test) / batch_size,
                              callbacks = callbacks_list)

Epoch 1/60
62/62 [==============================] - 111s - loss: 2.9196 - acc: 0.7749 - val_loss: 1.9296 - val_acc: 0.8440

KeyboardInterrupt: 

<br>
** Plot Accurracy**

<br>

In [ ]:
fig = plt.figure (figsize = (10,10))
ax1 = fig.add_subplot(1,1,1)
ax1.plot(history.history['acc'])
ax1.set_xlabel("epoch")
ax1.set_title('Validation Accuracy')
ax1.set_ylabel("accuracy")
plt.show()


<br>
### Export Results to File:

<br>
** 1 - Export Weights : **

<br>

In [ ]:
import h5py as h5py

model.save('vgg16.h5')


<br>
** 2 - Export Accurracy **

<br>

In [ ]:
# Conctruct Accurrracy Data Frame
Acc_vgg16 = pd.DataFrame(history.history['acc'] , columns = ['Accuracy'])

# Write to File
Acc_vgg16.to_csv('vgg16_acc.csv')